In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import db_lob as lob

In [3]:
# Step 1: load a list of messages
messages_file_path = '2505133.csv'
print("Reading messages...")
messages = lob.parse_FullMessages(messages_file_path)
print("Messages read!")

Reading messages...
Messages read!


In [4]:
# book
volume_threshold = 1000000
ticksize = 0.0001
book = lob.LimitOrderBook(volume_threshold, ticksize)

In [35]:
# Step 3: loop over the messages and feed them to the lob
# Just for testing, let's just look at the first N% messages.
pct = 0.5
n_msg = int(len(messages) * pct)
for msg in tqdm(messages[:n_msg], desc="Reconstructing the book"):
    maybe_bars = book.generic_incremental_update(msg)

    if maybe_bars is not None:
        vbars = pd.DataFrame.from_records(maybe_bars[0])
        fbars = pd.DataFrame.from_records(maybe_bars[1])

Reconstructing the book: 100%|████████| 824460/824460 [01:36<00:00, 8529.81it/s]


In [21]:
ask_side, bid_side = book.askTree, book.bidTree
a = list(ask_side.values())
a[0]

PriceLevel(price=15558000000, totalVolume=700000)

In [32]:
def df_from_book(book):
    import pandas as pd
    ask_side, bid_side = book.askTree, book.bidTree
    a = list(ask_side.values())
    b = list(bid_side.values())
    ask_price, bid_price, ask_volume, bid_volume = [], [], [], []
    for ask, bid in zip(a,b):
        ask_price.append(ask.price)
        bid_price.append(bid.price)
        ask_volume.append(ask.totalVolume)
        bid_volume.append(bid.totalVolume)
    df = pd.DataFrame(ask_volume, columns=['ask_volume'])
    df['ask_price'] = ask_price
    df['bid_volume'] = bid_volume
    df['bid_price'] = bid_price
    return df

In [33]:
df = df_from_book(book)
import os
dir = 'data_cleaned'
if os.path.isdir(dir)==False:
        os.mkdir(dir)
df.to_csv(dir+'/LOB.csv', index=False)